In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!unzip -q drive/'My Drive'/dataset-original.zip

In [3]:
!mkdir dataset-validation
!mkdir dataset-validation/cardboard
!mkdir dataset-validation/glass
!mkdir dataset-validation/paper
!mkdir dataset-validation/plastic
!mkdir dataset-validation/metal
!mkdir dataset-validation/trash
!ls
!ls dataset-original/trash

dataset-original  dataset-validation  drive  __MACOSX  sample_data
trash100.jpg  trash121.jpg  trash18.jpg  trash39.jpg  trash5.jpg   trash80.jpg
trash101.jpg  trash122.jpg  trash19.jpg  trash3.jpg   trash60.jpg  trash81.jpg
trash102.jpg  trash123.jpg  trash1.jpg	 trash40.jpg  trash61.jpg  trash82.jpg
trash103.jpg  trash124.jpg  trash20.jpg  trash41.jpg  trash62.jpg  trash83.jpg
trash104.jpg  trash125.jpg  trash21.jpg  trash42.jpg  trash63.jpg  trash84.jpg
trash105.jpg  trash126.jpg  trash22.jpg  trash43.jpg  trash64.jpg  trash85.jpg
trash106.jpg  trash127.jpg  trash23.jpg  trash44.jpg  trash65.jpg  trash86.jpg
trash107.jpg  trash128.jpg  trash24.jpg  trash45.jpg  trash66.jpg  trash87.jpg
trash108.jpg  trash129.jpg  trash25.jpg  trash46.jpg  trash67.jpg  trash88.jpg
trash109.jpg  trash12.jpg   trash26.jpg  trash47.jpg  trash68.jpg  trash89.jpg
trash10.jpg   trash130.jpg  trash27.jpg  trash48.jpg  trash69.jpg  trash8.jpg
trash110.jpg  trash131.jpg  trash28.jpg  trash49.jpg  trash6.jpg  

In [0]:
!shuf -n 101 -e dataset-original/glass/* | xargs -i mv {} dataset-validation/glass/
!shuf -n 144 -e dataset-original/paper/* | xargs -i mv {} dataset-validation/paper/
!shuf -n 53 -e dataset-original/cardboard/* | xargs -i mv {} dataset-validation/cardboard/
!shuf -n 82 -e dataset-original/plastic/* | xargs -i mv {} dataset-validation/plastic/
!shuf -n 60 -e dataset-original/metal/* | xargs -i mv {} dataset-validation/metal/
!shuf -n 37 -e dataset-original/trash/* | xargs -i mv {} dataset-validation/trash/

In [5]:
!ls dataset-original/glass | wc -l
!ls dataset-validation/glass | wc -l

400
101


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np

In [0]:
transformationsTrain = transforms.Compose([transforms.Resize(255), transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
                                           transforms.RandomRotation(degrees = (90, -90)), transforms.ToTensor(), 
                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transformationsValidation = transforms.Compose([transforms.Resize(255), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [0]:
train_set = datasets.ImageFolder("dataset-original", transform = transformationsTrain)
val_set = datasets.ImageFolder("dataset-validation", transform = transformationsValidation)

In [0]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=50, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size =50, shuffle=True)

In [10]:
model = torchvision.models.resnet34(pretrained=True)
model.cuda()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:05<00:00, 15.4MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
learningRate = 1e-3
epochs = 10
minibatches = 5

In [13]:
!pip install adabound
import adabound

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

[1,     5] loss: 4.667 accuracy: 0.560 itemLoss: 2.256
[1,    10] loss: 1.589 accuracy: 0.560 itemLoss: 1.666
[1,    15] loss: 1.290 accuracy: 0.400 itemLoss: 1.242
[1,    20] loss: 1.277 accuracy: 0.480 itemLoss: 1.211
[1,    25] loss: 1.184 accuracy: 0.680 itemLoss: 1.048
[1,    30] loss: 1.004 accuracy: 0.600 itemLoss: 1.112
[1,    35] loss: 0.935 accuracy: 0.700 itemLoss: 0.814
[1,    40] loss: 0.898 accuracy: 0.660 itemLoss: 0.802
Accuracy of the network on the validation images: 51 %
[2,     5] loss: 0.783 accuracy: 0.720 itemLoss: 0.839
[2,    10] loss: 0.829 accuracy: 0.680 itemLoss: 0.968
[2,    15] loss: 0.715 accuracy: 0.760 itemLoss: 0.749
[2,    20] loss: 0.890 accuracy: 0.700 itemLoss: 0.816
[2,    25] loss: 0.818 accuracy: 0.760 itemLoss: 0.601
[2,    30] loss: 0.733 accuracy: 0.760 itemLoss: 0.684
[2,    35] loss: 1.074 accuracy: 0.660 itemLoss: 0.864
[2,    40] loss: 0.845 accuracy: 0.700 itemLoss: 0.793
Accuracy of the network on the validation images: 39 %
[3,     5]

In [15]:
model = torchvision.models.resnet34(pretrained=True)
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

[1,     5] loss: 5.407 accuracy: 0.500 itemLoss: 3.394
[1,    10] loss: 1.652 accuracy: 0.600 itemLoss: 1.431
[1,    15] loss: 1.219 accuracy: 0.500 itemLoss: 1.239
[1,    20] loss: 1.179 accuracy: 0.640 itemLoss: 1.107
[1,    25] loss: 1.195 accuracy: 0.680 itemLoss: 1.096
[1,    30] loss: 1.057 accuracy: 0.600 itemLoss: 1.069
[1,    35] loss: 0.878 accuracy: 0.660 itemLoss: 0.832
[1,    40] loss: 1.069 accuracy: 0.540 itemLoss: 1.090
Accuracy of the network on the validation images: 62 %
[2,     5] loss: 1.010 accuracy: 0.580 itemLoss: 1.027
[2,    10] loss: 0.934 accuracy: 0.660 itemLoss: 0.964
[2,    15] loss: 0.750 accuracy: 0.640 itemLoss: 0.959
[2,    20] loss: 0.744 accuracy: 0.620 itemLoss: 1.030
[2,    25] loss: 0.777 accuracy: 0.700 itemLoss: 0.739
[2,    30] loss: 0.833 accuracy: 0.680 itemLoss: 1.008
[2,    35] loss: 0.679 accuracy: 0.700 itemLoss: 0.760
[2,    40] loss: 0.644 accuracy: 0.820 itemLoss: 0.607
Accuracy of the network on the validation images: 65 %
[3,     5]

In [13]:
torch.cuda.empty_cache()

model = torchvision.models.resnet152(pretrained=True)
model.cuda()

minibatches = 5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %f %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

RuntimeError: ignored

In [11]:
model = torchvision.models.densenet161(pretrained=True)
model.cuda()

minibatches = 5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %f %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:02<00:00, 48.6MB/s]


RuntimeError: ignored

In [0]:
model = torchvision.models.inception_v3(pretrained=True)
model.cuda()

minibatches = 5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %f %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')